# Clustering Crypto

In [61]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [62]:
# Use the following endpoint to fetch json data
import json
from pprint import pprint

url = "https://min-api.cryptocompare.com/data/all/coinlist"
res = requests.get(url)
res
data = res.json()
json_data = json.dumps(data)

In [63]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
df = pd.DataFrame(data["Data"])
df = df.T
url_request_df = df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
url_request_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN


In [64]:
df.keys()
# json file keys do not contain "TotalCoinSupply" column as in .csv file below. Further analysis will be using data from .csv file.

Index(['Algorithm', 'AssetLaunchDate', 'AssetTokenStatus', 'BlockNumber',
       'BlockReward', 'BlockTime', 'BuiltOn', 'CoinName', 'ContentCreatedOn',
       'DecimalPoints', 'Description', 'Difficulty', 'FullName', 'Id',
       'ImageUrl', 'IsTrading', 'IsUsedInDefi', 'MaxSupply', 'MktCapPenalty',
       'Name', 'NetHashesPerSecond', 'PlatformType', 'ProofType', 'Rating',
       'SmartContractAddress', 'SortOrder', 'Sponsored', 'Symbol', 'Taxonomy',
       'TotalCoinsMined', 'Url'],
      dtype='object')

In [65]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [66]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.set_index(crypto_df.columns[0])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [67]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading']==True]
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [68]:
# Keep only cryptocurrencies with a working algorithm
print(crypto_df['Algorithm'].isna())

Unnamed: 0
42       False
365      False
404      False
611      False
808      False
1337     False
2015     False
BTC      False
ETH      False
LTC      False
DASH     False
XMR      False
ETC      False
ZEC      False
BTS      False
DGB      False
BTCD     False
CRAIG    False
XBS      False
XPY      False
PRC      False
YBC      False
DANK     False
GIVE     False
KOBO     False
DT       False
CETI     False
SPR      False
WOLF     False
ACOIN    False
         ...  
AHT      False
RCC      False
OXY      False
KAPU     False
GRFT     False
MNZ      False
SCRM     False
LWF      False
ILT      False
ADM      False
UPX      False
TCH      False
TIP      False
DTEP     False
WAVES    False
PART     False
SLX      False
BTT      False
CHI      False
NXT      False
ZEPH     False
XQN      False
NETC     False
VPRC     False
GAP      False
SERO     False
UOS      False
BDX      False
ZEN      False
XBC      False
Name: Algorithm, Length: 1144, dtype: bool


In [69]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns="IsTrading")
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [70]:
# Remove rows with at least 1 null value
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [71]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.shape

(532, 5)

In [72]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna()
crypto_df.isna().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [73]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = crypto_df["CoinName"]
coin_name_df.head()

Unnamed: 0
42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [74]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns='CoinName')
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [75]:
# Create dummy variables for text features
crypto_df = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
# Standardize data
scaler = StandardScaler()
crypto_data_scaled = scaler.fit_transform(crypto_df)
print(crypto_data_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Reducing Dimensions Using PCA

In [92]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_data_scaled)

In [94]:
# Create a DataFrame with the principal components data
crypto_pca_df = pd.DataFrame(crypto_pca, columns=["principal component 1", "principal component 2", "principal component 3"])
crypto_pca

array([[-0.34161854,  1.04860319, -0.48008433],
       [-0.32493197,  1.0486666 , -0.48047973],
       [ 2.32568642,  1.55101999, -0.58278662],
       ...,
       [ 0.32166049, -2.31048861,  0.33270762],
       [-0.12042305, -2.09441739,  0.41567769],
       [-0.30146767,  0.87044626, -0.2514913 ]])

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [95]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [105]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)
# Fit the model
model.fit(crypto_pca)
# Predict clusters
predictions = model.predict(crypto_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
crypto_pca_df["class"] = model.labels_
crypto_pca_df.head()

,principal component 1,principal component 2,principal component 3,class
0,-0.341619,1.048603,-0.480084,0
1,-0.324932,1.048667,-0.480480,0
2,2.325686,1.551020,-0.582787,0
3,-0.150464,-1.318699,0.167734,1
4,-0.171700,-2.012957,0.299785,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [108]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_pca_df,
    x="principal component 3",
    y="principal component 2",
    z="principal component 1",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

:NdOverlay   [class]
   :Scatter   [principal component 1]   (principal component 2,class)

#### Table of Tradable Cryptocurrencies

In [41]:
# Table with tradable cryptos


In [42]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [43]:
# Scale data to create the scatter plot


In [44]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
